In [1]:
import pandas as pd
import numpy as np
import os
import re

In [ ]:
path_to_processed_data = r'C:\Niloofar\SCR\posture-prediction-models-main\external_data_processing\processed'
path_to_converted_data = r'C:\Niloofar\SCR\posture-prediction-models-main\external_data_processing\converted_data\s6'

In [3]:
column_names = ['LFHD', 'RFHD', 'LBHD', 'RBHD', 
                'C7', 'T10', 'CLAV', 'STRN',
                'RBAK', 'T12', 'S1', 'LSHO',
                'LUPA', 'LELB', 'LFRM', 'LWRA',
                'LWRB', 'LFIN', 'RSHO', 'RUPA',
                'RELB', 'RFRM', 'RWRA', 'RWRB',
                'RFIN', 'LASI', 'RASI', 'LPSA',
                'RPSI', 'LTHI', 'LKNE', 'LTIB', 
                'LANK', 'LHEE', 'LTOE', 'RTHI',
                'RKNE', 'RTIB', 'RANK', 'RHEE',
                'RTOE', ]

In [4]:
for item in os.listdir(path_to_processed_data):
    df = pd.read_csv(os.path.join(path_to_processed_data, item))
    # Convert the DataFrame to numpy array for easier reshaping
    data_array = df.to_numpy()

    # Separate x, y, z coordinates
    num_markers = 41
    num_frames = 101

    # Extract X, Y, Z segments
    x_coords = data_array[:, :num_markers]
    y_coords = data_array[:, num_markers:num_markers*2]
    z_coords = data_array[:, num_markers*2:]

    # Initialize the new array (303 rows: 3 rows per frame, 41 columns: markers)
    reshaped_data = np.zeros((num_frames * 3, num_markers))

   

    # Fill the reshaped_data array
    for i in range(num_frames):
        reshaped_data[i*3] = x_coords[i]    # x coordinates
        reshaped_data[i*3 + 1] = y_coords[i]  # y coordinates
        reshaped_data[i*3 + 2] = z_coords[i]  # z coordinates

    # Convert the reshaped numpy array back to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data, columns=column_names)

    # Form last input column

    W = 71
    H = 170

    match = re.search(r'technique_([^_]+)', item.split('.')[0])
    if match.group(1) == 'stoop':
        LT = 1

    if match.group(1) == 'semi-squat':
        LT = 2

    if match.group(1) == 'full-squat':
        LT = 3

    if item.split('.')[0].split('_')[-1] == '2':
        Load_X_Left = reshaped_df['LFIN'].iloc[-3]
        Load_X_Right = reshaped_df['RFIN'].iloc[-3]
        Load_X = (Load_X_Left + Load_X_Right) / 2

        Load_Y_Left = reshaped_df['LFIN'].iloc[-2]
        Load_Y_Right = reshaped_df['RFIN'].iloc[-2]
        Load_Y = (Load_Y_Left + Load_Y_Right) / 2

        Load_Z_Left = reshaped_df['LFIN'].iloc[-1]
        Load_Z_Right = reshaped_df['RFIN'].iloc[-1]
        Load_Z = (Load_Z_Left + Load_Z_Right) / 2

        HT = 2

       

    if item.split('.')[0].split('_')[-1] == '1':
        Load_X = reshaped_df['RFIN'].iloc[-3]
        Load_Y = reshaped_df['RFIN'].iloc[-2]
        Load_Z = reshaped_df['RFIN'].iloc[-1]
    
        HT = 1

    last_column = pd.Series([Load_X, Load_Y, Load_Z, LT, HT, W, H], name='Input')
    last_column = last_column.reindex(reshaped_df.index, fill_value=0)
    reshaped_df.insert(len(reshaped_df.columns), last_column.name, last_column)

    new_name = item.split('.')[0] + '_new.csv'
    reshaped_df.to_csv(os.path.join(path_to_converted_data, new_name), index=False, header=False)

    







In [5]:
item.split('.')[0]

'processed_L8_frames_4000-4280_technique_full-squat_hand_2'

In [6]:
reshaped_df['LFIN'].iloc[-1]

275.427246

In [7]:

match = re.search(r'technique_([^_]+)', item.split('.')[0])
match.group(1)

'full-squat'